<a href="https://www.kaggle.com/code/erayyuztyurk/diabetes-project?scriptVersionId=167702550" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# BUSINESS PROBLEM
------------------------------------------------------------------------------------------------------------------------
A machine learning model is requested to predict whether individuals have diabetes when their features are specified.
Before developing the model, it is expected that the necessary data analysis and feature engineering steps be performed.

# DATASET STORY
------------------------------------------------------------------------------------------------------------------------
The dataset is part of a large dataset maintained by the National Institute of Diabetes and Digestive and
Kidney Diseases in the United States. It consists of data used for diabetes research conducted on Pima Indian women
aged 21 and older living in Phoenix, the fifth-largest city in the state of Arizona, USA.

The target variable is defined as 'outcome,' where 1 indicates a positive diabetes test result, and 0 indicates
a negative result.

# FEATURES
------------------------------------------------------------------------------------------------------------------------
#### Variable Names
------------------------------------------------------------------------------------------------------------------------
- **Pregnancies:**               Number of pregnancies
- **Glucose:**                   2-hour plasma glucose concentration during an oral glucose tolerance test
- **Blood Pressure:**            Diastolic blood pressure (mm Hg)
- **Skin Thickness:**            Skinfold thickness (mm)
- **Insulin:**                   2-hour serum insulin (mu U/ml)
- **DiabetesPedigreeFunction:**  Diabetes pedigree function (2-hour plasma glucose concentration)
- **BMI:**                       Body mass index (weight in kg / (height in m)^2)
- **Age:**                       Age (years)
- **Outcome:**                   Presence (1) or absence (0) of diabetes
------------------------------------------------------------------------------------------------------------------------

# PROJECT TASKS

_The core focus of this project lies in feature engineering, where various techniques are applied to extract valuable insights from the dataset._

------------------------------------------------------------------------------------------------------------------------
#### Step 1: Exploratory Data Analysis
- 1.1 - Examining the big picture.
- 1.2 - Identifying numerical and categorical variables.
- 1.3 - Analyzing numerical and categorical variables.
- 1.4 - Conducting a target variable analysis.
- 1.5 - Performing outlier analysis.
- 1.6 - Performing missing data analysis.
- 1.7 - Performing correlation analysis.

#### Step 2: Feature Engineering
- 2.1 - Performing necessary operations for missing and outlier values.
- 2.2 - Creating new variables.
- 2.3 - Performing encoding operations.
- 2.4 - Standardizing numerical variables.

#### Step 3: Modelling
- 3.1 - K-Nearest Neighbors Model
- 3.2 - RandomForestClassifier Model

# Importing required libraries and setting options

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import KNNImputer
from sklearn.neighbors import LocalOutlierFactor
import missingno as msno

pd.set_option("display.float_format", lambda x: f"{x:.4f}")
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_rows",50)

def show_rows(show=False):
    if show:
        pd.set_option("display.max_rows", None)
    else:
        pd.set_option("display.max_rows", 50)

# Reading Dataset

In [ ]:
diabetes = pd.read_csv("/kaggle/input/diabetes/diabetes.csv")
df = diabetes.copy()
df

# Step 1: Exploratory Data Analysis
## 1.1 - Examine the big picture

In [ ]:
def check_df(dataframe, show_value_counts = False):
    import pandas as pd
    pd.set_option("display.colheader_justify","left")
    print("-------------------------------------------------------------------------------------------------------------------------------------------------")
    print("-------- SHAPE of Dataset -----------------------------------------------------------------------------------------------------------------------")
    print(dataframe.shape)
    print("-------------------------------------------------------------------------------------------------------------------------------------------------")
    print("-------- DATA TYPES of Dataset ------------------------------------------------------------------------------------------------------------------")
    print(dataframe.dtypes)
    print("-------------------------------------------------------------------------------------------------------------------------------------------------")
    print("-------- MEMORY USAGE of Dataset --------------------------------------------------------------------------------------------------------------  ")
    print(dataframe.memory_usage(deep=True) / (1024 * 1024), "\n (All are in MB)")
    print("-------------------------------------------------------------------------------------------------------------------------------------------------")
    print("-------- MISSING VALUES in Dataset --------------------------------------------------------------------------------------------------------------")
    print(dataframe.isnull().sum())
    print("-------------------------------------------------------------------------------------------------------------------------------------------------")
    print("-------- DESCRIPTIVE Info about Dataset ---------------------------------------------------------------------------------------------------------")
    print(dataframe.describe([0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]).T)
    print("-------------------------------------------------------------------------------------------------------------------------------------------------")

    if show_value_counts:
        print("-------- VALUE COUNTS in Dataset ----------------------------------------------------------------------------------------------------------------")
        for col in dataframe.columns:
            print(dataframe[col].value_counts())
            print("-------------------------------------------------------------------------------------------------------------------------------------------------")

In [ ]:
#- an overview for dataframe
check_df(df, show_value_counts=False)
#---------------------------------------------------------------------------------------------
target_label = "Outcome"
#---------------------------------------------------------------------------------------------

#### Comments:
- Target Label is "Outcome" and values are 0 and 1.
- There is no NaN values.
- There are many min 0 values which do not make sense and should not be 0 in normal circumstances (e.g. Glucose, BloodPressure, SkinThickness, Insulin, BMI)
- There are skewed variables. But it can be deceptive as min value is 0 for above columns.
- Kurtosis is not examined by choice (if desired, it is required statsmodels or manual calculation).
- Accepted ranges for variables should be checked online for more info.

## 1.2 - Identify numerical and categorical variables.

In [ ]:
def grab_column_names_based_on_types(dataframe, cardinal_threshold_percent=0.6, category_threshold_num=20):
    def is_date(string):
        from datetime import datetime as dt
        import pandas as pd
        try:
            if dt.strptime(string, '%Y-%m-%d %H:%M:%S'):
                return True
            else:
                return False
        except ValueError:
            return False
    date_cols = [col for col in dataframe.columns if (dataframe[col].dtype in ["O"] and all(is_date(str(value)) for value in dataframe.dropna(subset=[col])[col]) or type(dataframe[col].iloc[0]) is pd.Timestamp)]
    num_but_car_cols = [col for col in dataframe.columns if dataframe[col].dtype in ["int", "float"] and dataframe[col].nunique() == (dataframe.shape[0])]
    cat_but_car_cols = [col for col in dataframe.columns if dataframe[col].dtype in ["O"] and dataframe[col].nunique() > (dataframe.shape[0] * cardinal_threshold_percent) and not col in date_cols]
    car_cols = num_but_car_cols + cat_but_car_cols
    num_but_cat_cols = [col for col in dataframe.columns if dataframe[col].dtype in ["int", "float"] and dataframe[col].nunique() < category_threshold_num]
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtype in ["O"] and not col in cat_but_car_cols and not col in date_cols] + num_but_cat_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtype in ["int", "float"] and not col in num_but_cat_cols and not col in num_but_car_cols]

    print("------------------------------------------------------------------------------------------------------------")
    print(f"Observations Count: {dataframe.shape[0]}")
    print(f"Variables Count: {dataframe.shape[1]}")
    print("------------------------------------------------------------------------------------------------------------")
    print(f'Date Columns Count: {len(date_cols)} -> {date_cols}')
    print(f'Numeric Columns Count: {len(num_cols)} -> {num_cols}')
    print(f'Categorical Columns Count: {len(cat_cols)} -> {cat_cols}')
    print(f'Cardinal Columns Count: {len(car_cols)} -> {car_cols}')
    print("------------------------------------------------------------------------------------------------------------")
    print(f'Numeric but Cardinal Columns Count: {len(num_but_car_cols)} -> {num_but_car_cols}')
    print(f'Numeric but Categorical Columns Count: {len(num_but_cat_cols)} -> {num_but_cat_cols}')
    print(f'Categorical but Cardinal Columns Count: {len(num_but_cat_cols)} -> {cat_but_car_cols}')
    print("------------------------------------------------------------------------------------------------------------")
    if dataframe.shape[1] == (len(date_cols) + len(num_cols) + len(cat_cols) + len(car_cols)):
        print(f'Variables Count {dataframe.shape[1]} and Sum of Date,Numeric,Categorical and Cardinal Columns Counts {len(date_cols) + len(num_cols) + len(cat_cols) + len(car_cols)} are equal..')
    else:
        print(
            f'Variables Count {dataframe.shape[1]} and Sum of Date,Numeric,Categorical and Cardinal Columns Counts {len(date_cols) + len(num_cols) + len(cat_cols) + len(car_cols)} are NOT equal !!')
    print("------------------------------------------------------------------------------------------------------------")
    return num_cols, cat_cols, car_cols, date_cols

In [ ]:
 #- determine column datatypes
num_cols, cat_cols, car_cols, date_cols = grab_column_names_based_on_types(diabetes)
#---------------------------------------------------------------------------------------------

#### Comments:
- There are 7 numerical and 2 categorical columns
- Pregnancies is accepted as categorical.
- Target Label is "Outcome" and values are 0 and 1.

## 1.3 - Analyze numerical and categorical variables.

In [ ]:
def categoricals_summary(dataframe, column_name, plot=False):
    print(pd.DataFrame({column_name: dataframe[column_name].value_counts(),
                 "Ratio": 100 * (dataframe[column_name].value_counts() / len(dataframe))}))
    print("--------------------------------------------------------------------------------------------------------")

    if plot:
        sns.countplot(x=dataframe[column_name], data=dataframe)
        plt.show()
def numericals_summary(dataframe, column_name, plot = False, plot_bins = 20):

    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    print(dataframe[column_name].describe(quantiles).T)

    if plot:
        dataframe[column_name].hist(bins = plot_bins)
        plt.xlabel(column_name)
        plt.title(column_name)
        plt.show()

In [ ]:
 #- analysis of categorical variables
print("---------------------------------------------")
print("----- Analysis of Categorical Variables -----")
print("---------------------------------------------")
for col in cat_cols:
    print(f"----- {col} ---------------------------------------------------------------------")
    print(df.groupby(col).agg({target_label: ["mean", "count"]}))
    print("----------------------------------------------------------------------------------")
    categoricals_summary(df, col, plot=True)
#---------------------------------------------------------------------------------------------
 #- analysis of numerical variables
print("---------------------------------------------")
print("----- Analysis of Numerical Variables -------")
print("---------------------------------------------")
for col in num_cols:
    print(f"----- {col} ---------------------------------------------------------------------")
    print(df.groupby(target_label).agg({col: ["mean", "min", "max"]}))
    print("----------------------------------------------------------------------------------")
    numericals_summary(df, col, plot=True)
#---------------------------------------------------------------------------------------------
print("---------------------------------------------")
print("----- Boxplot Analysis ----------------------")
print("---------------------------------------------")
 #- box plots
for col in num_cols:
    print(f"----- {col} ---------------------------------------------------------------------")
    sns.boxplot(x=df[col],)
    plt.show(block=True)
    print("----------------------------------------------------------------------------------")

#### Comments:
- There are many variables with outliers.
- Pregnancies: Variable is right skewed. Most of the values are positioned between 0-2. It seems cases above 10 are rare.
- Outcome: In dataset, 1/3 of observations are as 1 and 2/3 of observation as 0 resulted.
- Glucose: It seems that variable almost normally distributed. (except zeroes)
- BloodPressure: It seems that variable is almost normally distributed. (except zeroes and approx. between 20-30)
- SkinThickness: It seems that variable is almost normally distributed. (except zeroes and approx. between 0-10 and above 55)
- Insulin: Variable is right skewed. Most of the values are positioned between 40-200. (except zeroes and approx. above 600)
- BMI: It seems that variable is almost normally distributed (except zeroes and approx. above 50).
- DiabetesPedigreeFunction: Variable is right skewed. Most of the values are positioned between 0.10-0.60. (except approx. above 1.50)
- Age: Variable is right skewed. Most of the values are positioned between 20-50. (except approx. above 60)

## 1.4 - Conduct a target variable analysis. (Mean of the target variable by categorical variables and mean of numerical variables by the target variable)

In [ ]:
 #- analysis of categorical variables
for col in cat_cols:
    print(f"{col} ---------------------------------------------------------------------------")
    df_grouped = df.groupby(col).agg({target_label: ["mean", "count"]})
    print(df_grouped.sort_values(by=(target_label, "mean"), ascending=False))
    print("----------------------------------------------------------------------------------")
#---------------------------------------------------------------------------------------------
 #- analysis of numerical variables
for col in num_cols:
    print(f"{col} ---------------------------------------------------------------------------")
    df_grouped = df.groupby(target_label, group_keys=False).agg({col: ["mean", "min", "max"]})
    print(df_grouped.sort_values(by=(col, "mean"), ascending=False))
    print("----------------------------------------------------------------------------------")
#---------------------------------------------------------------------------------------------

#### Comments:
##### Categorical Variables
- Pregnancies: 
-- Values between 0 and 3 have top frequencies in compare to others.
-- Values between 7 and 13 are distinctive for Outcome with over 40 percent (not considering counts).
-- Values between 14 and 17 are rare.
-- There can be created new features with 4 categories -> (0-3), (4-6), (7-13), (14-17)
- Outcome:     1/3 of observations are as 1 and 2/3 of observation as 0 resulted.

##### Numerical variables
- Glucose:     Glucose may be distinctive with 141.25 for 1 / 109.98 for 0. High glucose may cause positive result.
- Insulin:     Insulin may be distinctive with 100.34 for 1 / 68.79 for 0. High insulin may cause positive result.
- BMI:         BMI may be distinctive with 35.14 for 1 / 30.30 for 0. High BMI may cause positive result.

## 1.5 - Perform outlier analysis.

In [ ]:
def outlier_thresholds(dataframe, column_name, q1 = 0.25, q3 = 0.75, print_info=True):
    q1_num = dataframe[column_name].quantile(q1)
    q3_num = dataframe[column_name].quantile(q3)
    iqr = q3_num - q1_num
    lower_threshold = q1_num - iqr * 1.5
    upper_threshold = q3_num + iqr * 1.5
    if print_info:
        print(f"for {column_name}:")
        print(f"Q1 is {q1}")
        print(f"Q3 is {q3}")
        print(f"Q1 threshold is {q1_num}")
        print(f"Q3 threshold is {q3_num}")
        print(f"IQR is {iqr}")
        print(f"Lower threshold is {lower_threshold}")
        print(f"Upper Threshold is {upper_threshold}")
        print("-----------------------------------------------------------------------------------------------------------")
    return lower_threshold, upper_threshold
#----------------------------------------------------------------------------------------------------------------------
def check_if_outlier_exists(dataframe, column_name, q1 = 0.25, q3 = 0.75, print_details=True):
    low_limit, up_limit = outlier_thresholds(dataframe, column_name, q1, q3, print_details)
    if dataframe[((dataframe[column_name] < low_limit) | (dataframe[column_name] > up_limit))].shape[0] > 0:
        print(f"{column_name} : Outliers exist based on Q1={q1} and Q3={q3}")
        print(f"Under lower bound: {len(dataframe[dataframe[column_name] < low_limit])} observation of {len(dataframe)}")
        print(f"Above upper bound: {len(dataframe[dataframe[column_name] > up_limit])} observation of {len(dataframe)}")
        print("-------------------------------------------------------------------------------------------------------")
        return True
    else:
        print(f"{column_name} : No Outlier based on Q1={q1} and Q3={q3}")
        print("-------------------------------------------------------------------------------------------------------")
        return False
#----------------------------------------------------------------------------------------------------------------------
def grap_outliers(dataframe, column_name, q1 = 0.25, q3 = 0.75, get_incides=False, print_results=True):

    low_limit, up_limit = outlier_thresholds(dataframe, column_name, q1, q3)

    if print_results:
        if len(dataframe[(dataframe[column_name] < low_limit) | (dataframe[column_name] > up_limit)]) > 10:
            print(dataframe[(dataframe[column_name] < low_limit) | (dataframe[column_name] > up_limit)].head())
        else:
            print(dataframe[(dataframe[column_name] < low_limit) | (dataframe[column_name] > up_limit)])

    if get_incides:
        return dataframe[(dataframe[column_name] < low_limit) | (dataframe[column_name] > up_limit)].index
#----------------------------------------------------------------------------------------------------------------------
def rare_analyzer(dataframe, target_col_name, forCategoricals=True, forNumericals=False):
    if forCategoricals:
        for col in cat_cols:
            print(f'{col} Count: {len(dataframe[col])}')
            dff = pd.DataFrame({"Count": dataframe[col].value_counts(),
                                "Ratio": dataframe[col].value_counts() / len(dataframe),
                                "Target_Mean": dataframe.groupby(col)[target_col_name].mean()})
            print(dff.sort_values(by=["Ratio", "Target_Mean", "Count"], ascending=[False, False, False]))
            print("-----------------------------------------------------------------------------------")

    if forNumericals:
        for col in num_cols:
            print(f'{col} Count: {len(dataframe[col])}')
            dff = pd.DataFrame({"Count": dataframe[col].value_counts(),
                                "Ratio": dataframe[col].value_counts() / len(dataframe),
                                "Target_Mean": dataframe.groupby(col)[target_col_name].mean()})
            print(dff.sort_values(by=["Ratio", "Target_Mean", "Count"], ascending=[False, False, False]))
            print("-----------------------------------------------------------------------------------")
#----------------------------------------------------------------------------------------------------------------------
def local_outlier_factor_analysis(dataframe, plot=False, return_scores= False, n_neighbors=20):
    lof = LocalOutlierFactor(n_neighbors=n_neighbors)
    lof.fit_predict(dataframe)
    scores_neg = np.sort(lof.negative_outlier_factor_)
    if plot:
        pd.DataFrame(scores_neg).plot(stacked=True, xlim = [0,30], style=".-")
        plt.show()
    if return_scores:
        return scores_neg

In [ ]:
 #- box plots
print("---------------------------------------------")
print("----- Boxplot Analysis ----------------------")
print("---------------------------------------------")

for col in num_cols:
    print(f"----- {col} ---------------------------------------------------------------------")
    sns.boxplot(x=df[col],)
    plt.show(block=True)
    print("----------------------------------------------------------------------------------")
#---------------------------------------------------------------------------------------------
 #- check if any outlier exists
print("---------------------------------------------")
print("----- Outlier Analysis ----------------------")
print("---------------------------------------------")

for col in df.columns:
    check_if_outlier_exists(df, col, q1=0.25, q3=0.75)
#---------------------------------------------------------------------------------------------

 #- check rarity of values
print("---------------------------------------------")
print("----- Rare Analysis -------------------------")
print("---------------------------------------------")
rare_analyzer(df, target_label, forNumericals=True)
#---------------------------------------------------------------------------------------------
print("---------------------------------------------")
print("----- LOF Analysis --------------------------")
print("---------------------------------------------")
 #- local factor analysis
scores = local_outlier_factor_analysis(df, plot=True, return_scores=True)
print(f" Based on LOF scores, threshold can be selected as {scores[10]}")

#### Comments:
- There are many outliers if thresholds are chosen as 0.25 for Q1 and 0.75 for Q3.
- They should be checked again after correction of 0 values in some features.
- It can be created rare variable for Pregnancies between 14-17
- for LOF, it seems that 10. value can be used as threshold for now (-1.7062264314830031)

## 1.6 - Perform missing data analysis.

In [ ]:
def missing_values_summary(dataframe, print_df=True, return_na_col_names=False,return_na_df=False):
    na_cols = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    na_list = dataframe[na_cols].isnull().sum().sort_values(ascending=False)
    na_ratios = (dataframe[na_cols].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    na_df = pd.DataFrame(na_ratios).reset_index().rename(columns={"index": "NA_Column", 0: "Ratio"})
    if print_df:
        if len(na_df) == 0:
            print("There is no missing values in the dataset.")
        else:
            print(na_df)

    if return_na_col_names:
        return na_cols

    if return_na_df:
        return na_df

In [ ]:
 #- check for missing values
missing_values_summary(df)
#---------------------------------------------------------------------------------------------

#### Comments:
- There is no missing values in the dataset but 0 values should be considered as NaN, then all should be checked again.

## 1.7 - Perform correlation analysis.

In [ ]:
def correlation_matrix_for_numerical_variables(dataframe, numeric_columns, show_plot=True, return_corr_matrix=False):
    print("-------- Correlation Mapping ---------")
    print("Very Weak Correlation:   (0.00 - 0.19)")
    print("Weak Correlation:        (0.20 - 0.39)")
    print("Moderate Correlation:    (0.40 - 0.59)")
    print("Strong Correlation:      (0.60 - 0.79)")
    print("Very Strong Correlation: (0.80 - 1.00)")
    print("--------------------------------------")
    corr_matrix = dataframe[numeric_columns].corr()
    if show_plot:
        sns.heatmap(corr_matrix, vmin=-1, vmax=1, cmap="RdBu", annot=True)
        plt.show()
    if return_corr_matrix:
        return corr_matrix

In [ ]:
 #- create correlation matrix of numerical variables
correlation_matrix_for_numerical_variables(df, num_cols, show_plot=True)
#---------------------------------------------------------------------------------------------

#### Comments:
- It seems that there is only a moderate positive correlation between "SkinThickess" and "Insulin" with 0.44.

# STEP 2: Feature Engineering
## 2.1 - Perform necessary operations for missing and outlier values. 

_Although there are no missing observations in the dataset, observations containing 0 values in variables like Glucose, Insulin, etc. may represent missing values. For example, a person's Glucose or Insulin value cannot be 0. Considering this, we can assign NaN to zero values in the relevant variables and then apply operations to missing values._

In [ ]:
def missing_values_summary(dataframe, print_df=True, return_na_col_names=False,return_na_df=False):
    print("---------------------------------------------")
    print("----- Missing Values Summary ----------------")
    print("---------------------------------------------")
    
    na_cols = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    na_list = dataframe[na_cols].isnull().sum().sort_values(ascending=False)
    na_ratios = (dataframe[na_cols].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    na_df = pd.DataFrame(na_ratios).reset_index().rename(columns={"index": "NA_Column", 0: "Ratio"})
    if print_df:
        if len(na_df) == 0:
            print("There is no missing values in the dataset.")
        else:
            na_df
    
    if return_na_col_names:
        return na_cols

    if return_na_df:
        return na_df
#----------------------------------------------------------------------------------------------------------------------
def missing_values_table(dataframe, na_name=False):
    print("---------------------------------------------")
    print("----- Missing Values Table ------------------")
    print("---------------------------------------------")
    
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    missing_df
    if na_name:
        return na_columns
#----------------------------------------------------------------------------------------------------------------------
def missing_vs_target_analysis(dataframe, target_var_name):
    print("---------------------------------------------")
    print("----- Missing Values vs. Target Analysis ----")
    print("---------------------------------------------")
    
    temp_df = dataframe.copy()
    na_columns = missing_values_summary(temp_df, return_na_col_names=True)

    for col in na_columns:
        temp_df[col + "_na_flag"] = np.where(temp_df[col].isnull(), 1, 0)

    na_flags = temp_df.loc[:, temp_df.columns.str.contains("_na_flag")].columns

    for col in na_flags:
        print(pd.DataFrame({"Target_Mean": temp_df.groupby(col)[target_var_name].mean(),
                            "Count": temp_df.groupby(col)[target_var_name].count()}))
        print("----------------------------------------------------------------")
#----------------------------------------------------------------------------------------------------------------------
def missing_corr_analysis(dataframe, plot_show=True):
    temp_df = dataframe.copy()
    na_columns = missing_values_summary(temp_df, return_na_col_names=True)
    for col in na_columns:
        temp_df[col + "_na"] = np.where(temp_df[col].isnull(), 1, 0)
    na_flags = temp_df.loc[:, temp_df.columns.str.contains("_na")].columns
    corr_missing = temp_df[na_flags].corr()
    #print(corr_missing)
    if plot_show:
        sns.heatmap(corr_missing, cmap="RdBu", annot=True, vmax=1, vmin=-1)
        plt.show(block=True)
#----------------------------------------------------------------------------------------------------------------------
def remove_outliers(dataframe, col_name, q1=0.25, q3=0.75):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name, q1, q3)
    df_without_outliers = dataframe[~((df[col_name] < low_limit) | (dataframe[col_name] > up_limit))]
    return df_without_outliers
#----------------------------------------------------------------------------------------------------------------------
def winsorize_with_thresholds(dataframe, col_name, q1=0.25, q3=0.75):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name, q1, q3)
    dataframe.loc[(dataframe[col_name] < low_limit), col_name] = low_limit
    dataframe.loc[(dataframe[col_name] > up_limit), col_name] = up_limit
#----------------------------------------------------------------------------------------------------------------------
def local_outlier_factor_analysis(dataframe, plot=False, return_scores= False, n_neighbors=20):
    lof = LocalOutlierFactor(n_neighbors=n_neighbors)
    lof.fit_predict(dataframe)
    scores_neg = np.sort(lof.negative_outlier_factor_)
    if plot:
        pd.DataFrame(scores_neg).plot(stacked=True, xlim = [0,30], style=".-")
        plt.show()
    if return_scores:
        return scores_neg

#### Copy Dataset

In [ ]:
df_w = df.copy()

#### Missing Values

In [ ]:
 #- assigning NaN of 0 values of "Glucose, BloodPressure, SkinThickness, Insulin, BMI" are corrected
zero_removing_columns = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction"]
df_w[zero_removing_columns] = df_w[zero_removing_columns].replace(0, np.nan)
df_w

In [ ]:
#---------------------------------------------------------------------------------------------
 #- check missing values
missing_vs_target_analysis(df_w, target_label)
print("--------------------------------------------------------------------------------------")
#---------------------------------------------------------------------------------------------
missing_corr_analysis(df_w) #-> Significant correlation between SkinThickness and Insulin

#### Comments:
- It seems that there is a moderate positive correlation between "SkinThickess" and "Insulin" with 0.66 for missing values.

#### Missing Values Solutions

In [ ]:
 #- assign means of Pregnancies for each
for col in num_cols:
    df_w[col].fillna(df_w.groupby("Pregnancies")[col].transform("mean"), inplace=True)

df_w

#### Comments:
- All missing values are filled by their means considering Pregnancy count.

#### Outliers

In [ ]:
 #- box plots
for col in df_w.columns:
    if col != target_label:
        sns.boxplot(x=df_w[col],)
        plt.show(block=True)
 #- analysis of categorical variables after corrections
for col in cat_cols:
    print(f"{col} ---------------------------------------------------------------------------")
    df_grouped = df.groupby(col).agg({"Outcome": ["mean", "count"]})
    print(df_grouped.sort_values(by=("Outcome", "mean"), ascending=False))
    print("----------------------------------------------------------------------------------")
 #- analysis of numerical variables after corrections
for col in num_cols:
    print(f"{col} ---------------------------------------------------------------------------")
    df_grouped = df.groupby("Outcome", group_keys=False).agg({col: ["mean", "min", "max"]})
    print(df_grouped.sort_values(by=(col, "mean"), ascending=False))
    print("----------------------------------------------------------------------------------")
 #- check if any outlier exists
for col in df.columns:
    check_if_outlier_exists(df, col, q1=0.05, q3=0.95) #--> seem proper as Q1 and Q3
#---------------------------------------------------------------------------------------------
 #- local factor analysis
scores = local_outlier_factor_analysis(df_w, plot=True, return_scores=True) #-> 5. value seems proper (-1.80269)
df_w[scores < scores[5]] #-> only 5 values which can be eliminated (5/768)
df_w[scores < scores[5]].describe().T


#### Outliers Solutions


In [ ]:
#---------------------------------------------------------------------------------------------
 #- winsorizing outliers
for col in df.columns:
    winsorize_with_thresholds(df_w, col, q1=0.1, q3=0.9)
#---------------------------------------------------------------------------------------------
 #- eliminating outliers according to LOF
df_w = df_w.drop(index=df_w[scores < scores[5]].index)
print(f"New dataset size: {df_w.shape[0]}")

#### Comments:
- 0 values of "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction" are corrected as NaN.
- There is a correlation of missing values between SkinThickness and Insulin (0.66).
- Missing values are filled based on Pregnancies group Means. KNN Imputer has not been chosen as Insulin (48%) and SkinThickness (30%) are significantly empty.
- Q1 and Q3 for Outliers are selected as 0.1 and 0.9 as it does not touch that much to many outliers but insulin.
-   Winsorisation has been applied.
- 5 observation of 768 are removed from dataset which are found via LOF.

## 2.2 - Create new variables.

In [ ]:
#---------------------------------------------------------------------------------------------
rare_analyzer(df, target_label, forNumericals=True)
#---------------------------------------------------------------------------------------------
df_w["Pregnancies_levels"] = pd.cut(df_w["Pregnancies"], bins=[0,3,6,13,17],labels=["Ordinary","More than Ordinary","Rare","Extraordinary"])
df_w["BMI_levels"] = pd.cut(df_w["BMI"], bins=[0,18.5,24.99999,29.99999,90], labels=["Underweight","Normal Weight","Overweight","Obese"])
df_w["Age_levels"] = pd.cut(df["Age"], bins=[0,10,20,30,40,50,60,70,80,90,100,110,120])
df_w["Glucose_levels"] = pd.cut(df["Glucose"], bins=[0,115,180,280,380,1000], labels=["Excellent","Good","Action Suggested","See Doctor","Dangerous"])
df_w


#### Comments:
- from Pregnancies, (0-3), (4-6), (7-13), (14-17) are created.
- from BMI (<0.185)-> Underweight, (0.185-0.249999)-> Normal Weight, (0.25-0.2999999)-> Overweight, (> 30)-> Obese
- from Age (0,10,20,30,40,50,60,70,80,90,100,110,120)
- form Glucose (<150), (150-199),(200-249),(250-299),(300-349), (>=350)

### 2.3 - Perform encoding operations.

In [ ]:
def label_encoder_binary(dataframe, return_col_names=True, return_encoded_dataframe=False):
    if return_encoded_dataframe:
        return_col_names = False
    df_ = dataframe.copy()
    binary_columns = []
    label_encoder = LabelEncoder()
    binary_columns = [col for col in df_.columns if df_[col].dtype not in [int, float] and df_[col].nunique() == 2]
    if len(binary_columns) > 0:
        for col in binary_columns:
            df_[col] = label_encoder.fit_transform(df_[col])
    if return_col_names:
        return binary_columns
    if return_encoded_dataframe:
        return df_
#-----------------------------------------------------------------------------------------------------------------------
def one_hot_encoder(dataframe, categorical_columns, drop_first=True, return_dataframe=False, print_df=False):
    df_ = dataframe.copy()
    df_ = pd.get_dummies(df_, columns=categorical_columns, drop_first=drop_first)
    
    if print_df:
        print(df_.head())
    if return_dataframe:
        return df_

In [ ]:
#---------------------------------------------------------------------------------------------
 #- use one hot encoder
ohe_cols = ["Pregnancies_levels","BMI_levels","Age_levels","Glucose_levels","Pregnancies"]
df_w = one_hot_encoder(df_w,ohe_cols, return_dataframe=True)
df_w

#### Comments:
- As there is no binary variable except target, one hot encoder has been applied.

## 2.4 - Standardize numerical variables.

In [ ]:
#---------------------------------------------------------------------------------------------
standard_cols = [col for col in df_w.columns if col not in ohe_cols]
scaler = StandardScaler()
scaler.fit_transform(df_w[standard_cols])
df_w

# STEP 3: Modelling

Now, we test our processed data on two different models.

## 3.1 - K-Nearest Neighbors Model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score

dff = df.copy()
dff.dropna(inplace=True)

X = dff.drop(target_label, axis=1)
y = dff[target_label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

knn = KNeighborsClassifier(n_neighbors=5)
kfold = 10
scores = cross_val_score(knn, X_train, y_train, cv=kfold)

print("----- RAW DATASET RESULTS -----")
#print(f"--> Scores: {scores}")
print("--------------------------------------------------------------------------------------")
print(f"KNN with {kfold}-Fold CV Score: {scores.mean():.4f}")
print("--------------------------------------------------------------------------------------")
print("\n")

#---------------------------------------------------------------------------------------------
X = df_w.drop(target_label, axis=1)
y = df_w[target_label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

knn = KNeighborsClassifier(n_neighbors=5)
kfold = 10
scores = cross_val_score(knn, X_train, y_train, cv=kfold)

print("----- PROCESSED DATASET RESULTS -----")
#print(f"--> Scores: {scores}")
print("--------------------------------------------------------------------------------------")
print(f"KNN with {kfold}-Fold CV Score: {scores.mean():.4f}")
print("--------------------------------------------------------------------------------------")

## 3.2 - Random Forest Classifier Model

In [ ]:
def feature_importance_levels (model, features, num=len(X), save=False):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features.columns})
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",ascending=False)[0: num])
    plt.title('Features') 
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

dff = df.copy()
dff.dropna(inplace=True)

y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

scores = []
kfold = 10

for _ in range(kfold):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
    rf = RandomForestClassifier().fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    scores.append(accuracy_score(y_pred, y_test))

print("----- RAW DATASET RESULTS -----")
#print(f"--> Scores: {scores}")
print("--------------------------------------------------------------------------------------")
print(f"Random Forest with {kfold}-Fold CV Score: {np.mean(scores):.4f}")
print("--------------------------------------------------------------------------------------")
print("\n")
feature_importance_levels(rf, X_train)
#---------------------------------------------------------------------------------------------
y = df_w["Outcome"]
X = df_w.drop(["Outcome"], axis=1)

scores = []
kfold = 10

for _ in range(kfold):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
    rf = RandomForestClassifier().fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    scores.append(accuracy_score(y_pred, y_test))

print("----- PROCESSED DATASET RESULTS -----")
#print(f"--> Scores: {scores}")
print("--------------------------------------------------------------------------------------")
print(f"Random Forest with {kfold}-Fold CV Score: {np.mean(scores):.4f}")
print("--------------------------------------------------------------------------------------")
feature_importance_levels(rf, X_train)

More models can be applied. As feature engineering is the main focus of this project, only 2 models is applied for demonstration purposes.

---
### I hope you found the analysis insightful and informative! 
### Your feedback is greatly appreciated, and I welcome any suggestions for improvement. Feel free to reach out with any questions or comments.
---